# EXAMEN 2 mod2 

## Importar

In [1]:
# %load basic
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns',200)
pd.options.display.float_format = '{:.2f}'.format
liga = '/home/ef/Documents/Diplomado/EXAMEN0202/Copia de '

In [2]:
df = pd.read_csv(liga + 'summary.csv')
genre = pd.read_csv(liga + 'song_genres.csv')
feat = pd.read_csv(liga + 'audio_features.csv')
lyrics = pd.read_csv(liga + 'song_lyrics.csv')

In [3]:
## Se une con género, audio features y lyrics
df = df.merge(genre, 
              on = 'track_id',
              how = 'left').merge(feat,
                                  on = 'track_id',
                                  how = 'left').merge(lyrics, 
                                                      on = 'track_id',
                                                      how = 'left')

In [4]:
song = pd.read_csv(liga + 'song_play_count.csv')
## Se agrupa el promedio por canción
song = pd.DataFrame(song.groupby('song_id')['play_count'].mean()).reset_index()
## Se une con summary
df = df.merge(song, on = 'song_id', how = 'left')

In [5]:
tags = pd.read_csv(liga + 'artist_tags.csv')
terms = pd.read_csv(liga + 'artist_terms.csv')
loc = pd.read_csv(liga + 'artist_location.csv')

## Se une con summary
df = df.merge(tags, 
              on = 'artist_id', 
              how = 'left').merge(terms, 
                                  on = 'artist_id', 
                                  how = 'left').merge(loc, 
                                                      on = 'artist_id',
                                                      how = 'left')


## Análisis exploratorio

In [6]:
## Se omiten las columnas sin varianza
df = df[[x for x in df.columns if x not in ['energy',
                                            'analysis_sample_rate_x',
                                            'analysis_sample_rate_y',
                                            'sample_rate', 
                                            'equal_loudness', 
                                            'lossless',
                                            'artist']]].copy()

In [7]:
words = [x for x in df.columns if x in lyrics.columns[1:]]
df[words] = df[words].fillna(-1)
df[['longitude','latitude','play_count']] = df[['longitude',
                                                'latitude',
                                                'play_count']].fillna(0)
df[['main_genre','location','mbtag']] = df[['main_genre',
                                            'location',
                                            'mbtag']].fillna('No_aplica')

In [8]:
df = df[~df['happiness'].isnull()].copy()
df.reset_index(drop = True, inplace = True)
pd.DataFrame(df.isnull().sum() / len(df)
            ).sort_values(by = 0,
                          ascending = False).head(5)

,0
term,0.00
track_id,0.00
of,0.00
need,0.00
never,0.00


In [9]:
## Se genera lista de géneros por default
generos = list(pd.DataFrame(df['main_genre'].value_counts(1)).reset_index().iloc[1:,0])
generos = [x.lower() for x in generos]

## Columnas con la unión de tags y terms
df['buscar'] = df['mbtag'].astype(str) + df['term'].astype(str) + ['fin fin']

## Se busca en la columna creada el primer género mencionado
aux = []
for i in range(len(df)):
    if df['main_genre'][i] == 'No_aplica':
        apoyo = [x for x in df['buscar'][i].split() if x in generos] + ['No_aplica']
        apoyo = apoyo[0].lower()
        aux.append(apoyo)
    else:
        aux.append(df['main_genre'][i])

## Se reemplaza la columna
df['main_genre'] = aux
## Se vuelven minúsculas todas las letras
df['main_genre'] = [x.lower() for x in df['main_genre']]
## Ahora existen muy pocas con "No aplica" (nulos)
df['main_genre'].value_counts(1)

rock         0.39
pop          0.15
electronic   0.09
metal        0.08
jazz         0.08
rap          0.04
punk         0.03
blues        0.03
folk         0.02
country      0.02
reggae       0.02
rnb          0.02
latin        0.01
world        0.01
no_aplica    0.00
new age      0.00
Name: main_genre, dtype: float64

## ETL

### Crear funciones

In [10]:
def longitud(df,col):
    df[f'{col}_long'] = df[col].str.len()
    df[f'{col}_n_words'] = df[col].str.split(' ').str.len()
    df[f'{col}_n_letters'] = df[col].map(lambda x:sum(map(str.isalpha, x)))
    return df

### Aplicarlas

In [11]:
df = df[[x for x in df.columns if x not in ['track_id', 'song_id', 'artist_id',
                                            'codec', 'downmix', 'md5_encoded',
                                            'location', 'buscar', 'longitude', 'latitude']]]

num = [x for x in list(df.describe().columns) if x not in words]
cat = list(df[[x for x in df.columns if x not in num + words]].columns)
df[cat] = df[cat].astype(str)
df = df[cat + num + words].copy()

In [12]:
## Se aplica las funciones y algunas extra
for col in ['title', 'artist_name', 'release', 'mbtag', 'term']:
    df = longitud(df,col)

### Outliers

In [13]:
## Función para omitir outliers mediante IQR
def outlier(df, x , p = 0.2):
    var = df[x]
    q1 = var.quantile(p/2) #acota el % indicado, tanto máx como mín
    q3 = var.quantile(1 - p/2)
    iqr = q3 - q1

    df = df[(var.isnull()) | ((var >= q1 - 1.5*iqr) & (var <= q3 + 1.5*iqr))].copy()
    df.reset_index(drop = True, inplace = True)
    return df

In [14]:
display(df[num].describe([.01,.1,.9,.99]))
dim_antes = len(df)

for col in num:
    df = outlier(df,col, p = 0.22)
    
display('{:.2%}'.format(len(df) / dim_antes))  
df[num].describe([.01,.1,.9,.99])

,mode_confidence,end_of_fade_in,key_confidence,year,duration,time_signature_confidence,start_of_fade_out,time_signature,tempo,artist_hotttnesss,key,loudness,bit_rate,length,replay_gain,danceability,femaleness,acousticness,aggressiveness,electronicity,happiness,festiveness,relaxation,sadness,brightness,tonality,insrumentalness,play_count
count,37400.00,37400.00,37400.00,37400.00,37400.00,37400.00,37400.00,37400.00,37400.00,37400.00,37400.00,37400.00,37400.00,37400.00,37400.00,37400.00,37400.00,37400.00,37400.00,37400.00,37400.00,37400.00,37400.00,37400.00,37400.00,37400.00,37400.00,37400.00
mean,0.47,0.93,0.44,1996.67,248.82,0.52,239.69,3.63,125.20,0.47,5.35,-9.72,110715.74,44.99,-10.78,0.43,0.54,0.31,0.38,0.47,0.46,0.41,0.47,0.37,0.60,0.54,0.25,1.33
std,0.19,2.19,0.28,11.69,109.82,0.37,107.97,1.20,34.43,0.12,3.61,4.86,28612.50,16.06,3.79,0.38,0.32,0.36,0.38,0.32,0.28,0.31,0.36,0.31,0.36,0.38,0.35,1.86
min,0.00,0.00,0.00,1927.00,0.78,0.00,0.78,0.00,0.00,0.00,0.00,-52.25,51080.00,0.99,-21.99,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1%,0.00,0.00,0.00,1960.00,44.93,0.00,44.02,1.00,49.59,0.00,0.00,-25.66,64028.00,29.54,-16.75,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00
10%,0.23,0.00,0.03,1979.00,141.19,0.00,133.43,1.00,86.09,0.35,0.00,-16.12,64028.00,30.04,-14.79,0.01,0.08,0.00,0.01,0.07,0.08,0.02,0.03,0.03,0.05,0.02,0.00,0.00
50%,0.48,0.20,0.46,2000.00,232.33,0.56,222.80,4.00,123.01,0.46,6.00,-8.66,128056.00,34.04,-11.55,0.33,0.57,0.10,0.22,0.43,0.45,0.38,0.43,0.28,0.70,0.61,0.05,1.12
90%,0.71,3.07,0.80,2008.00,372.35,1.00,360.79,4.00,170.21,0.60,10.00,-4.63,128111.00,60.08,-5.82,0.99,0.97,0.93,0.98,0.94,0.86,0.85,0.97,0.87,0.99,0.99,0.93,3.11
99%,0.88,9.39,1.00,2010.00,620.04,1.00,601.32,7.00,219.76,0.82,11.00,-2.81,128111.00,60.13,1.23,1.00,1.00,1.00,1.00,1.00,0.99,0.97,1.00,0.97,1.00,1.00,1.00,6.33
max,1.00,60.51,1.00,2010.00,2342.77,1.00,2190.43,7.00,268.09,1.08,11.00,2.45,320000.00,348.00,38.40,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,75.00


'95.07%'

,mode_confidence,end_of_fade_in,key_confidence,year,duration,time_signature_confidence,start_of_fade_out,time_signature,tempo,artist_hotttnesss,key,loudness,bit_rate,length,replay_gain,danceability,femaleness,acousticness,aggressiveness,electronicity,happiness,festiveness,relaxation,sadness,brightness,tonality,insrumentalness,play_count
count,35558.00,35558.00,35558.00,35558.00,35558.00,35558.00,35558.00,35558.00,35558.00,35558.00,35558.00,35558.00,35558.00,35558.00,35558.00,35558.00,35558.00,35558.00,35558.00,35558.00,35558.00,35558.00,35558.00,35558.00,35558.00,35558.00,35558.00,35558.00
mean,0.47,0.72,0.44,1996.63,243.80,0.52,234.77,3.62,125.33,0.47,5.33,-9.61,110774.61,44.76,-10.87,0.43,0.54,0.31,0.38,0.47,0.46,0.41,0.47,0.37,0.60,0.55,0.25,1.27
std,0.19,1.31,0.27,11.64,95.77,0.37,94.00,1.19,34.35,0.11,3.60,4.68,28544.16,15.06,3.61,0.38,0.32,0.36,0.38,0.32,0.28,0.31,0.36,0.31,0.36,0.38,0.34,1.39
min,0.00,0.00,0.00,1938.00,0.78,0.00,0.78,0.00,0.00,0.01,0.00,-31.77,51080.00,0.99,-21.99,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1%,0.00,0.00,0.00,1960.00,46.05,0.00,44.56,1.00,51.54,0.26,0.00,-24.53,64028.00,29.54,-16.75,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00
10%,0.23,0.00,0.03,1979.00,140.80,0.00,133.17,1.00,86.23,0.35,0.00,-15.94,64039.00,30.04,-14.81,0.01,0.08,0.00,0.01,0.07,0.09,0.02,0.03,0.03,0.05,0.02,0.00,0.00
50%,0.48,0.20,0.46,2000.00,230.63,0.56,221.31,4.00,123.08,0.46,5.00,-8.58,128056.00,33.16,-11.58,0.33,0.56,0.10,0.22,0.43,0.45,0.38,0.42,0.28,0.71,0.62,0.05,1.12
90%,0.71,2.72,0.80,2008.00,362.97,1.00,352.65,4.00,170.63,0.59,10.00,-4.62,128111.00,60.08,-6.04,0.99,0.97,0.93,0.98,0.94,0.86,0.85,0.97,0.87,0.99,0.99,0.91,3.04
99%,0.88,5.87,1.00,2010.00,547.13,1.00,539.12,7.00,219.35,0.80,11.00,-2.78,128111.00,60.13,0.22,1.00,1.00,1.00,1.00,1.00,0.98,0.97,1.00,0.97,1.00,1.00,1.00,5.46
max,1.00,7.15,1.00,2010.00,674.74,1.00,654.96,7.00,268.09,0.92,11.00,2.45,224055.00,85.73,6.15,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,7.50


In [15]:
cat = ['main_genre']

## Se despliegan las posibles opciones para cada variable categórica,
# agrupando las categorías con menos de cierto % de frecuencia
adj = [] 
for col in cat:
    aux = []
    aux = df[col].value_counts(1, dropna = False).reset_index()
    global adj
    for i in range(len(aux)):
        if aux.iloc[i,1] < .02:
            adj.append(aux.iloc[i,0])

# adj será la lista de elementos p reemplazar por 'Otros', importante para ETL
df[cat] = df[cat].replace(adj, 'Otros')
df['main_genre'].value_counts(1)

rock         0.39
pop          0.16
metal        0.09
electronic   0.08
jazz         0.08
rap          0.04
punk         0.04
blues        0.02
Otros        0.02
folk         0.02
country      0.02
reggae       0.02
rnb          0.02
Name: main_genre, dtype: float64

## Modelado "continuo"

In [16]:
## Crear rangos para predecir
df['OBJETIVO'] , bins = pd.qcut(df['duration'].astype(int),
                             q = 5,
                             duplicates = 'drop', 
                             retbins = True)
df['OBJETIVO'] = df['OBJETIVO'].astype(str)
df['OBJETIVO'] = [int(x[-5:][:2]) for x in df['OBJETIVO']]
df['OBJETIVO'].value_counts(1)

50   0.20
13   0.20
72   0.20
74   0.20
6    0.20
Name: OBJETIVO, dtype: float64

In [17]:
num = [x for x in num if x not in ['duration']]

In [18]:
X = df[cat + num + words]
y = df['OBJETIVO'].values.ravel()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.9)

### OHE

In [19]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse = False)

## Se obtienen las dummies de cat y se une con num (por eso hereda el index)
X_train = pd.DataFrame(ohe.fit_transform(X_train[cat]),
                       columns = list(ohe.get_feature_names(X[cat].columns)), 
                       index = X_train.index).join(X_train[num + words])

## Sólo transform
X_test = pd.DataFrame(ohe.transform(X_test[cat]),
                      columns = list(ohe.get_feature_names(X[cat].columns)), 
                      index = X_test.index).join(X_test[num + words])

### Modelos

In [20]:
from sklearn.ensemble import RandomForestClassifier 
forest = RandomForestClassifier()

param_forest = {'n_estimators': [x for x in range(100, 1500, 100)],
                'max_features': ['auto', 'sqrt', 'log2'],
                'criterion': ['gini', 'entropy'],
                'class_weight': ['balanced', None],
                'min_samples_split': [x for x in range(2, 50, 2)],
                'min_samples_leaf': [x/100 for x in range(5, 55, 5)]
               }

from sklearn.model_selection import RandomizedSearchCV
search_forest = RandomizedSearchCV(param_distributions = param_forest, 
                            cv = 4, 
                            n_jobs = -1, 
                            scoring = 'accuracy',
                            estimator = forest,
                            verbose = 5,
                            n_iter = 10)

search_forest.fit(X_train,y_train)

Fitting 4 folds for each of 10 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  5.6min finished


RandomizedSearchCV(cv=4, estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'class_weight': ['balanced', None],
                                        'criterion': ['gini', 'entropy'],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [0.05, 0.1, 0.15,
                                                             0.2, 0.25, 0.3,
                                                             0.35, 0.4, 0.45,
                                                             0.5],
                                        'min_samples_split': [2, 4, 6, 8, 10,
                                                              12, 14, 16, 18,
                                                              20, 22, 24, 26,
                                                              28, 30, 32, 34,
                           

In [21]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier()

param_ada={'n_estimators':[50,100,200],
           'learning_rate':[.001,0.01,.1,.5]}

search_ada = RandomizedSearchCV(param_distributions = param_ada, 
                                cv = 4, 
                                n_jobs = -1, 
                                scoring = 'accuracy', 
                                estimator = ada, 
                                verbose = 5,
                                n_iter = 9)

search_ada.fit(X_train,y_train)

Fitting 4 folds for each of 9 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed: 11.3min finished


RandomizedSearchCV(cv=4, estimator=AdaBoostClassifier(), n_iter=9, n_jobs=-1,
                   param_distributions={'learning_rate': [0.001, 0.01, 0.1,
                                                          0.5],
                                        'n_estimators': [50, 100, 200]},
                   scoring='accuracy', verbose=5)

In [22]:
from xgboost.sklearn import XGBClassifier
xgb = XGBClassifier(learning_rate = 0.1, 
                    n_estimators = 190,
                    max_depth = 5, 
                    min_child_weight = 2,
                    objective = 'binary:logistic',
                    subsample = 0.9, 
                    colsample_bytree = 0.8, 
                    seed = 22)

xgb.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=2, missing=nan, monotone_constraints='()',
              n_estimators=190, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=22, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, seed=22, subsample=0.9,
              tree_method='exact', validate_parameters=1, verbosity=None)

### Voting

In [23]:
from sklearn.ensemble import VotingClassifier
vc = VotingClassifier(estimators = [('Forest', search_forest.best_estimator_), 
                                    ('ADA', search_ada.best_estimator_),
                                    ('XGB', xgb)], 
                      voting = 'soft')

vc.fit(X_train, y_train)

VotingClassifier(estimators=[('Forest',
                              RandomForestClassifier(class_weight='balanced',
                                                     criterion='entropy',
                                                     min_samples_leaf=0.1,
                                                     min_samples_split=24,
                                                     n_estimators=800)),
                             ('ADA',
                              AdaBoostClassifier(learning_rate=0.001,
                                                 n_estimators=200)),
                             ('XGB',
                              XGBClassifier(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=0.8, gamma...
                                            interaction_constraints='',
                  

In [24]:
def score_metrics(X, y, estimator):
    from sklearn.model_selection import cross_val_score
    ls_scores_roc = cross_val_score(estimator = estimator,
                                    X = X, y = y, 
                                    scoring = 'accuracy', 
                                    n_jobs = -1, 
                                    cv = 4)
    print(f'Accuracy media: {np.mean(ls_scores_roc):,.2f}, desviación estándar: {np.std(ls_scores_roc)}')

In [25]:
score_metrics(X = X_train, 
              y = y_train, 
              estimator = vc)

Accuracy media: 0.97, desviación estándar: 0.0017623183699282357


In [26]:
scores = pd.DataFrame()
scores['Modelo'] = [x[0] for x in vc.estimators] + ['Voting']
scores['Train'] = [x[1].score(X_train, y_train) for x in vc.estimators] + [vc.score(X_train,
                                                                                    y_train)]
scores['Test'] = [x[1].score(X_test, y_test) for x in vc.estimators] + [vc.score(X_test,
                                                                                    y_test)]
scores

,Modelo,Train,Test
0,Forest,0.59,0.57
1,ADA,0.93,0.92
2,XGB,0.99,0.97
3,Voting,0.99,0.97


In [27]:
## Así explican las mejores 10 variables a la target, del mejor modelo antes de VC
pd.DataFrame(zip(X_train.columns, xgb.feature_importances_),
             columns = ['Variable', 'Relevancia']).sort_values(by = 'Relevancia',
                                                               ascending = False).head(10).reset_index(drop = True)

,Variable,Relevancia
0,start_of_fade_out,0.24
1,main_genre_electronic,0.04
2,main_genre_punk,0.03
3,see,0.01
4,so,0.01
5,was,0.01
6,get,0.01
7,main_genre_metal,0.01
8,caus,0.01
9,the,0.01


In [28]:
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(y_true = y_test, y_pred = vc.predict(X_test))/len(y_test), 
             index = list(sorted(np.unique(y_test))), 
             columns = list(sorted(np.unique(y_test))))

,6,13,50,72,74
6,0.19,0.00,0.01,0.00,0.00
13,0.00,0.19,0.01,0.00,0.00
50,0.00,0.01,0.19,0.00,0.00
72,0.00,0.00,0.00,0.19,0.00
74,0.00,0.00,0.00,0.00,0.21


## Modelado discreto

In [54]:
df['OBJETIVO2'] = (df['main_genre'] == 'rock')*1
df['OBJETIVO2'].value_counts(1)

0   0.61
1   0.39
Name: OBJETIVO2, dtype: float64

In [55]:
X = df[cat + num]
y = df['OBJETIVO2'].values.ravel()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.9)

### OHE

In [56]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse = False)

## Se obtienen las dummies de cat y se une con num (por eso hereda el index)
X_train = pd.DataFrame(ohe.fit_transform(X_train[cat]),
                       columns = list(ohe.get_feature_names(X[cat].columns)), 
                       index = X_train.index).join(X_train[num])

## Sólo transform
X_test = pd.DataFrame(ohe.transform(X_test[cat]),
                      columns = list(ohe.get_feature_names(X[cat].columns)), 
                      index = X_test.index).join(X_test[num])

### Modelos

In [59]:
from sklearn.ensemble import RandomForestClassifier 
forest = RandomForestClassifier()

param_forest = {'n_estimators': [x for x in range(100, 1500, 100)],
                'max_features': ['auto', 'sqrt', 'log2'],
                'criterion': ['gini', 'entropy'],
                'class_weight': ['balanced', None],
                'min_samples_split': [x for x in range(2, 50, 2)],
                'min_samples_leaf': [x/100 for x in range(5, 55, 5)]
               }

from sklearn.model_selection import RandomizedSearchCV
search_forest = RandomizedSearchCV(param_distributions = param_forest, 
                            cv = 4, 
                            n_jobs = -1, 
                            scoring = 'roc_auc',
                            estimator = forest,
                            verbose = 5,
                            n_iter = 10)

search_forest.fit(X_train,y_train)

Fitting 4 folds for each of 10 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  8.2min finished


RandomizedSearchCV(cv=4, estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'class_weight': ['balanced', None],
                                        'criterion': ['gini', 'entropy'],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [0.05, 0.1, 0.15,
                                                             0.2, 0.25, 0.3,
                                                             0.35, 0.4, 0.45,
                                                             0.5],
                                        'min_samples_split': [2, 4, 6, 8, 10,
                                                              12, 14, 16, 18,
                                                              20, 22, 24, 26,
                                                              28, 30, 32, 34,
                           

In [60]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier()

param_ada={'n_estimators':[50,100,200],
           'learning_rate':[.001,0.01,.1,.5]}

search_ada = RandomizedSearchCV(param_distributions = param_ada, 
                                cv = 4, 
                                n_jobs = -1, 
                                scoring = 'roc_auc', 
                                estimator = ada, 
                                verbose = 5,
                                n_iter = 9)

search_ada.fit(X_train,y_train)

Fitting 4 folds for each of 9 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:  6.3min finished


RandomizedSearchCV(cv=4, estimator=AdaBoostClassifier(), n_iter=9, n_jobs=-1,
                   param_distributions={'learning_rate': [0.001, 0.01, 0.1,
                                                          0.5],
                                        'n_estimators': [50, 100, 200]},
                   scoring='roc_auc', verbose=5)

In [61]:
from xgboost.sklearn import XGBClassifier
xgb = XGBClassifier(learning_rate = 0.1, 
                    n_estimators = 190,
                    max_depth = 5, 
                    min_child_weight = 2,
                    objective = 'binary:logistic',
                    subsample = 0.9, 
                    colsample_bytree = 0.8, 
                    seed = 22)

xgb.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=2, missing=nan, monotone_constraints='()',
              n_estimators=190, n_jobs=0, num_parallel_tree=1, random_state=22,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=22,
              subsample=0.9, tree_method='exact', validate_parameters=1,
              verbosity=None)

### Voting

In [63]:
from sklearn.ensemble import VotingClassifier
vc = VotingClassifier(estimators = [('Forest', search_forest.best_estimator_), 
                                    ('ADA', search_ada.best_estimator_),
                                    ('XGB', xgb)], 
                      voting = 'soft')

vc.fit(X_train, y_train)

VotingClassifier(estimators=[('Forest',
                              RandomForestClassifier(class_weight='balanced',
                                                     max_features='sqrt',
                                                     min_samples_leaf=0.05,
                                                     min_samples_split=40,
                                                     n_estimators=1300)),
                             ('ADA',
                              AdaBoostClassifier(learning_rate=0.1,
                                                 n_estimators=100)),
                             ('XGB',
                              XGBClassifier(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=0.8, gamma=0,
                                            gpu_id=-1, importance_type='gain',
           

In [64]:
def score_metrics_roc(X, y, estimator):
    from sklearn.model_selection import cross_val_score
    ls_scores_roc = cross_val_score(estimator = estimator,
                                    X = X, y = y, 
                                    scoring = 'roc_auc', 
                                    n_jobs = -1, 
                                    cv = 4)
    print(f'ROC media: {np.mean(ls_scores_roc):,.2f}, desviación estándar: {np.std(ls_scores_roc)}')

In [65]:
score_metrics_roc(X = X_train, 
              y = y_train, 
              estimator = vc)

ROC media: 1.00, desviación estándar: 0.0


In [66]:
scores = pd.DataFrame()
scores['Modelo'] = [x[0] for x in vc.estimators] + ['Voting']
scores['Train'] = [x[1].score(X_train, y_train) for x in vc.estimators] + [vc.score(X_train,
                                                                                    y_train)]
scores['Test'] = [x[1].score(X_test, y_test) for x in vc.estimators] + [vc.score(X_test,
                                                                                    y_test)]
scores

,Modelo,Train,Test
0,Forest,0.76,0.77
1,ADA,1.00,1.00
2,XGB,1.00,1.00
3,Voting,1.00,1.00


In [67]:
## Así explican las mejores 10 variables a la target, del mejor modelo antes de VC
pd.DataFrame(zip(X_train.columns, xgb.feature_importances_),
             columns = ['Variable', 'Relevancia']).sort_values(by = 'Relevancia',
                                                               ascending = False).head(10).reset_index(drop = True)

,Variable,Relevancia
0,electronicity,0.15
1,main_genre_pop,0.11
2,danceability,0.10
3,main_genre_metal,0.09
4,main_genre_jazz,0.08
5,main_genre_electronic,0.08
6,main_genre_punk,0.06
7,main_genre_rap,0.05
8,main_genre_blues,0.05
9,main_genre_Otros,0.04


In [68]:
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(y_true = y_test, y_pred = vc.predict(X_test))/len(y_test), 
             index = list(sorted(np.unique(y_test))), 
             columns = list(sorted(np.unique(y_test))))

,0,1
0,0.61,0.00
1,0.00,0.39


## Predict

### Lectura

In [ ]:
val = pd.read_csv(liga + 'EXAMEN_VAL.csv')
val.sample(3)

### Funciones

In [ ]:
val[cat] = val[cat].replace(adj, 'Otros')

In [ ]:
X_val = val[cat + num]
X_val.sample(3)

### Mejor modelo

In [ ]:
## El mejor modelo fue el ensamble en paralelo
scores

In [ ]:
import pickle

## Guardar modelo
with open('examen_2_2.pkl', 'wb') as file:
    pickle.dump(vc, file)

## Abrir un modelo
with open('examen_2_2.pkl', 'rb') as file:
    modelo = pickle.load(file)

## Listo para usarse
modelo

In [ ]:
## Se aplica el One Hot Encoder para los dummies de variables categóricas
X_val = pd.DataFrame(ohe.fit_transform(X_val[cat]),
                       columns = list(ohe.get_feature_names(X[cat].columns)), 
                       index = X_val.index).join(X_val[num + words])
X_val.sample(3)

In [ ]:
## Hay columnas en este nuevo set que el modelo nunca ha visto
aux = [x for x in X_val.columns if x not in X_train.columns]
aux

In [ ]:
## Dado que son registros nunca antes vistos, se omiten
X_val = X_val[[x for x in X_val.columns if x not in aux]].copy()
[x for x in X_val.columns if x not in X_train.columns]

### Resultado

In [ ]:
## Se reemplazan missings por la media del entrenamiento
val['VARIABLE_ANTES'] = val['VARIABLE_ANTES'].replace(np.nan,
                                                      X_train['VARIABLE_ANTES'].mean()).astype(int)

## Se usan los mismos bines definidos en entrenamiento
val['OBJETIVO'] = pd.cut(val['VARIABLE_ANTES'],
                             bins = bins).astype(str)

## Sólo nos quedamos con el entero de la cota superior
val['OBJETIVO'] = [int(x[-5:][:2]) for x in val['OBJETIVO']]
val['OBJETIVO'].value_counts(1)

In [ ]:
resultado = pd.DataFrame(val['OBJETIVO'],columns = ['real'])
resultado['estimado'] = modelo.predict(X_val)

## Buen modelo!
from sklearn.metrics import confusion_matrix
cm = pd.DataFrame(confusion_matrix(y_true = resultado['real'],
                                   y_pred = resultado['estimado'])/len(resultado), 
                  index = list(sorted(np.unique(y_test))), 
                  columns = list(sorted(np.unique(y_test))))
display(cm)

## Con buena acertividad (suma de diagonal en la matriz de confusión)
'{:.2%}'.format(np.asarray(cm).trace())

In [69]:
## Tono para cuando termina código
from IPython.lib.display import Audio
import numpy as np

framerate = 4410
play_time_seconds = 1

t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)
audio_data = np.sin(5*np.pi*300*t) + np.sin(2*np.pi*240*t)

## La siguiente línea debe ir debajo del código p que suene
Audio(audio_data, rate=framerate, autoplay=True)